In [1]:
#!/usr/bin/env python3

# Copyright (c) Facebook, Inc. and its affiliates.
# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.

from mephisto.abstractions.databases.local_database import LocalMephistoDB
from mephisto.tools.data_browser import DataBrowser as MephistoDataBrowser
from mephisto.data_model.worker import Worker
from mephisto.data_model.unit import Unit
from collections import Counter
from model_chat_blueprint import *
import pandas as pd

db = LocalMephistoDB()
mephisto_data_browser = MephistoDataBrowser(db=db)

DO_REVIEW = True

# task_names = ["model_chat"]
task_names = ["commonsense_model_chat_pilot"]

units = []
for t in task_names:
    new_units = mephisto_data_browser.get_units_for_task_name(t)
    print(t)
    print(Counter([u.get_status() for u in new_units]))
    units.extend(new_units)

print(f"prev len: {len(units)}")
print(Counter([u.get_status() for u in units]))

print(units[0])
unit = units[0]
message_data = []
r_keys = ['inconsistent_setting', 'inconsistent_action', 'disfluent', 'none_all_good']
for unit in units:
    data = mephisto_data_browser.get_data_from_unit(unit)
    submission_data = data['data']['final_submission']['all_ratings']
    # print(submission_data.keys())
    action = submission_data['messages']['2'][1]
    response = submission_data['messages']['3'][1]
    ratings = [submission_data['checkboxValues']['3'].get(k, None) for k in r_keys]
    error_text = submission_data['textFieldValues'].get('3', {}).get('why_error', None)
    better_narration = submission_data['textFieldValues'].get('3', {}).get('better_narration', None)

    # print(submission_data)
    message_data.append([action, response, *ratings, error_text, better_narration])

df = pd.DataFrame(message_data, columns=["action", "response", *r_keys, "error_text", "better_narration"])

print(df)

for c in ['action', 'response', 'error_text', 'better_narration']:
    df[c] = df[c].str.wrap(30)
print(df)
# print(data)
# print()
# print(data.keys())

commonsense_model_chat_pilot
Counter({'completed': 10})
prev len: 10
Counter({'completed': 10})
MTurkUnit(24748, 3G9UA71JVXIZ4E7LSGQQYV4XK567JB, completed)
                                              action  \
0                              What makes you happy?   
1  I just wish I knew how to have more fun and ge...   
2  I say: "dear, could you hand me that ore stone...   
3                      How was your day, good smith?   
4  I pick up the sword and brandish it and say I ...   
5  A take a seat and say, after our feast we will...   
6             You give the arrowhead to the Soldier.   
7                            You thank the Soldier.    
8                 Ask the trainer to feed the horse.   
9                           Walk to training course.   

                                            response  inconsistent_setting  \
0  The trainer is a man who loves to train his ho...                 False   
1  The sad townsman is walking by giving up all t...                 Fa

In [2]:
df

,action,response,inconsistent_setting,inconsistent_action,disfluent,none_all_good,error_text,better_narration
0,What makes you happy?,The trainer is a man who loves\nto train his h...,False,False,False,True,I am talking with the sad\ntownsman.,"As a trainer, nothing brings\nme more joy than..."
1,I just wish I knew how to have\nmore fun and g...,The sad townsman is walking by\ngiving up all ...,False,False,False,True,None,"Having given up all hope, the\nsad townsman si..."
2,"I say: ""dear, could you hand\nme that ore ston...",The smith gave you an ore\nstone.,True,False,False,False,"I'm the smith, I can't give\nmyself a an ore s...",The wife gives you an ore\nstone
3,"How was your day, good smith?","You asked the the smith, ""how\nwas your day, g...",False,False,False,True,None,None
4,I pick up the sword and\nbrandish it and say I...,I am ready to defend the\ntribe. I brandish a ...,False,False,False,True,None,None
5,"A take a seat and say, after\nour feast we wil...","You take a seat and say, after\nour feast we w...",False,False,False,True,None,None
6,You give the arrowhead to the\nSoldier.,You gave the arrowhead to the\nsoldier.,False,False,False,True,None,None
7,You thank the Soldier.,You thank the soldier.,False,False,False,True,None,None
8,Ask the trainer to feed the\nhorse.,You asked the trainer to feed\nthe horse.,False,False,False,True,None,You asked the trainer to feed\nthe horse and h...
9,Walk to training course.,The noble walked to the\ntraining course.,False,False,False,True,None,None


In [ ]:
for line in message_data:
    print("-"*100)
    print(line[0], "->", line[1])
    action, response, *ratings, error_text, better_narration

In [6]:
message_data = []
r_keys = ['inconsistent_setting', 'inconsistent_action', 'disfluent', 'none_all_good']
for unit in units:
    data = mephisto_data_browser.get_data_from_unit(unit)
    submission_data = data['data']['final_submission']['all_ratings']
    # print(submission_data.keys())
    action = submission_data['messages']['2'][1]
    response = submission_data['messages']['3'][1]
    ratings = [submission_data['checkboxValues']['3'].get(k, None) for k in r_keys]
    error_text = submission_data['textFieldValues'].get('3', {}).get('why_error', None)
    better_narration = submission_data['textFieldValues'].get('3', {}).get('better_narration', None)
    if better_narration is None:
        continue
    print("-"*100)
    print(action, "->", response)
    true_ratings = []
    for i, r in enumerate(ratings):
        if r == True:
            true_ratings.append(r_keys[i])
    print(f"Rated: {','.join(true_ratings)}")
    print(f"Why Error: {error_text}")
    print(f"Better Narration: {better_narration}")

----------------------------------------------------------------------------------------------------
What makes you happy? -> The trainer is a man who loves to train his horses.
Rated: none_all_good
Why Error: I am talking with the sad townsman.
Better Narration: As a trainer, nothing brings me more joy than training horses.
----------------------------------------------------------------------------------------------------
I just wish I knew how to have more fun and get the ladies to notice me. -> The sad townsman is walking by giving up all their hope.
Rated: none_all_good
Why Error: None
Better Narration: Having given up all hope, the sad townsman sighs, shrugs his shoulders, and walks away.
----------------------------------------------------------------------------------------------------
I say: "dear, could you hand me that ore stone? I need to smelt it." -> The smith gave you an ore stone.
Rated: inconsistent_setting
Why Error: I'm the smith, I can't give myself a an ore stone
B